In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import math
import keras.backend as K
import csv
from tqdm import tqdm
import re
from PIL import Image
import PIL


2021-12-25 23:06:59.120041: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-25 23:06:59.120127: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Frequency


In [2]:
words_frq = []
frq = []
word_frq = []
sum_frq = 0
with open("Datasets/meaningful_words_with_frq.csv", 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if len(row[0]) > 20: 
            continue
        words_frq.append([row[0], int(row[1])])
        frq.append(int(row[1]))
        sum_frq += int(row[1])


In [3]:
words_frq.sort(key = lambda x: x[1], reverse=True)


In [4]:
hfw = words_frq[:int(len(words_frq)/2)]
lfw = words_frq[int(len(words_frq)/2):]
highf_w = [item[0] for item in hfw]
lowf_w = [item[0] for item in lfw]
hf = frq[:int(len(words_frq)/2)]
lf = frq[int(len(words_frq)/2):]
ref = lf[0]
print(ref)

329153


## Multiply images by Frq

Run this section only if you want to make extra images for hf words in your dataset for the first time

In [ ]:
entries = os.listdir('ImageDataset/TrainSet/word/')
trainwords = []
trainwords_path = []
for e in entries:
    trainwords.append(e.split('_')[1].split('.')[0])
    trainwords_path.append('ImageDataset/TrainSet/word/'+e)
    
entries = os.listdir('ImageDataset/TestSet/word/')
testwords = []
testwords_path = []
for e in entries:
    testwords.append(e.split('_')[1].split('.')[0])
    testwords_path.append('ImageDataset/TestSet/word/'+e)

In [ ]:
num_extras = []
hf_count = 0
for w in tqdm(trainwords):
    if w in highf_w:
        hf_count += 1
        index = highf_w.index(w)
        f = hf[index]
        num_extras.append(int((int(f/ref)-1)/70))
        
print(f"# hf: {hf_count}  # lf: {len(trainwords)-hf_count}")

In [ ]:
total = 0
for i in num_extras:
    total+=i
print(f"Total number of extra images {total}")

In [ ]:
# train set
pbar = tqdm(total = total)
for w, p, n in tqdm(zip(trainwords, trainwords_path, num_extras)):
    im1 = Image.open(p)
    im2 = im1.copy()
    new_path = p.split('.')[0]
    for i in range(n):
            im2.save(new_path+"."+str(i)+'.png', "PNG")
    pbar.update(n)

# CNN

In [5]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)
batch_size = 32
train_dataset = train.flow_from_directory("ImageDataset/TrainSet/",
                                          target_size=(30,150),
                                          batch_size = batch_size,
                                          class_mode = 'categorical')
                                         
test_dataset = test.flow_from_directory("ImageDataset/TestSet/",
                                          target_size=(30,150),
                                          batch_size = batch_size,
                                          class_mode = 'categorical')

Found 11876 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [6]:
test_dataset.class_indices


{'nonword': 0, 'word': 1}

### Network Structure 1

In [18]:
model = tf.keras.models.Sequential()

model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(30,150,3)))
model.add(keras.layers.MaxPool2D(2,2))
# model.add(keras.layers.BatchNormalization())


model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
# model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
# model.add(keras.layers.BatchNormalization())


model.add(keras.layers.Flatten())


model.add(keras.layers.Dense(128,activation='relu'))
# model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Dense(64,activation='relu'))

model.add(keras.layers.Dense(8,activation='relu'))

# model.add(keras.layers.Dense(1,activation='sigmoid'))
model.add(keras.layers.Dense(2, activation='softmax'))


### Compile

In [19]:
model.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 28, 148, 32)       896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 14, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 12, 72, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 6, 36, 64)        0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 6, 36, 64)         0         
                                                                 
 conv2d_14 (Conv2D)          (None, 4, 34, 128)       

In [20]:

steps_per_epoch = len(train_dataset)

model.fit(train_dataset,
         steps_per_epoch = steps_per_epoch,
         epochs = 10,
         validation_data = test_dataset
       
         )

Epoch 1/10
372/372 [==============================] - 32s 86ms/step - loss: 0.5533 - accuracy: 0.6841 - val_loss: 0.4955 - val_accuracy: 0.7462
Epoch 2/10
372/372 [==============================] - 32s 85ms/step - loss: 0.3853 - accuracy: 0.8141 - val_loss: 0.3868 - val_accuracy: 0.8202
Epoch 3/10
372/372 [==============================] - 32s 86ms/step - loss: 0.3180 - accuracy: 0.8553 - val_loss: 0.3418 - val_accuracy: 0.8432
Epoch 4/10
372/372 [==============================] - 32s 87ms/step - loss: 0.2757 - accuracy: 0.8771 - val_loss: 0.3039 - val_accuracy: 0.8602
Epoch 5/10
372/372 [==============================] - 32s 87ms/step - loss: 0.2412 - accuracy: 0.8911 - val_loss: 0.3056 - val_accuracy: 0.8565
Epoch 6/10
372/372 [==============================] - 32s 87ms/step - loss: 0.2135 - accuracy: 0.9051 - val_loss: 0.2848 - val_accuracy: 0.8758
Epoch 7/10
372/372 [==============================] - 33s 87ms/step - loss: 0.1856 - accuracy: 0.9202 - val_loss: 0.3058 - val_accuracy:

In [21]:
import pickle
outfile = open("check_points/CNN_model",'wb')
pickle.dump(model,outfile)
outfile.close()

2021-12-25 23:41:30.330576: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://6a08ce4b-4868-4d7a-8007-788b306a5421/assets


## Train Set Results

In [22]:


entries = os.listdir('ImageDataset/TrainSet/nonword/')
nonwords = []
for e in entries:
    nonwords.append(e)


    
entries = os.listdir('ImageDataset/TrainSet/word/')
words = []
for e in entries:
    words.append(e)

In [23]:
prob = []
all_set = []
for e in tqdm(nonwords):
    word = (e.split('_'))[1].split('.')[0]
    if word in all_set:
        continue
    all_set.append(word)
    filename = "ImageDataset/TrainSet/nonword/" + e
    img = image.load_img(filename,target_size=(30,150))
 
    Y = image.img_to_array(img)
    X = np.expand_dims(Y,axis=0)
    X = X/255
    val = model.predict(X)
    prob.append([word, round(float(val[0][0]), 6), round(float(val[0][1]), 6),0])
    


100%|███████████████████████████████████████| 5000/5000 [03:02<00:00, 27.38it/s]


In [24]:
for e in tqdm(words):
    word = (e.split('_'))[1].split('.')[0]
    if word in all_set:
        continue
    all_set.append(word)
    filename = "ImageDataset/TrainSet/word/" + e
    img = image.load_img(filename,target_size=(30,150))
 
    Y = image.img_to_array(img)
    X = np.expand_dims(Y,axis=0)
    X = X/255
    val = model.predict(X)
    prob.append([word, round(float(val[0][0]), 6), round(float(val[0][1]), 6),1])



100%|███████████████████████████████████████| 6876/6876 [03:04<00:00, 37.29it/s]


In [25]:
with open("Datasets/train_dataset_prob.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    for k in tqdm(prob):
        writer.writerow(k)

100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 586402.71it/s]


In [26]:
with open("Datasets/train_dataset_unique_words.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    for k in tqdm(prob):
        writer.writerow([k[0], k[3]])

100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 324556.15it/s]
